In [ ]:
# !pip install torchmetrics

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
from torchmetrics.functional import sacre_bleu_score

from io import open
import unicodedata
import string
import re
import random
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Dataset
# https://download.pytorch.org/tutorial/data.zip

In [ ]:
# Encoding text based on index. I.e one hot encoding for each unique word

# start and end of sentence tokens
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2  #current no of words is 2, SOS and EOS

    def addSentence(self, sentence):
        for w in sentence.split(" "):
            self.addWord(w)

    def addWord(self, w):
        self.word2index[w] = self.word2index.get(w, self.n_words)
        # self.word2count[w] = 1
        self.index2word[self.n_words] = w
        self.n_words += 1



In [ ]:
# COnvert unicode to ascii
def unicodeToAscii(s):
    return "".join(c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != "Mn")


# lowercase, trim and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
# split the file into lines, and then split the lines into pairs of languages
# The reverse flag is used for french to english translation

def getSeparetedData(lang1, lang2, reverse=False):
    
    text_file = open("%s-%s.txt"  % (lang1, lang2), "r", encoding='utf-8')
    lines = text_file.read().strip().split('\n')

    # split on tab
    pairs = [[normalizeString(s) for s in line.split('\t')] for line in lines]
    print(pairs[0:10])

    # if reverse is true, reverse the pairs
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        in_lang = Lang(lang2)
        out_lang = Lang(lang1) 
    else:
        in_lang = Lang(lang1)
        out_lang = Lang(lang2)

    return in_lang, out_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):

    # read the file and separete the data and create Lang objects
    in_lang, out_lang, pairs = getSeparetedData(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))

    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))

    print("Counting words...")
    for pair in pairs:
        in_lang.addSentence(pair[0])
        out_lang.addSentence(pair[1])
    print("Counted words.")
    print(in_lang.name, in_lang.n_words)
    print(out_lang.name, out_lang.n_words)

    return in_lang, out_lang, pairs


In [ ]:
in_lang, out_lang, pairs = prepareData('eng', 'fra')
print(random.choice(pairs))

[['go .', 'va !'], ['run !', 'cours !'], ['run !', 'courez !'], ['wow !', 'ca alors !'], ['fire !', 'au feu !'], ['help !', 'a l aide !'], ['jump .', 'saute .'], ['stop !', 'ca suffit !'], ['stop !', 'stop !'], ['stop !', 'arrete toi !']]
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words.
eng 63973
fra 65674
['we re going to need some help .', 'nous allons avoir besoin d aide .']


In [ ]:
# Defining the encoder class
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)


    def forward(self, input, hidden):
        # reshape embedding vector as batch size=1, sequence length=1 and embedding size=embedding_size 
        embedded = self.embedding(input)
        # print("ENCODER: ")
        # print(embedded.shape)
        # the GRU layer that comes next expects an input of size [sequence_length, batch_size, input_size]
        output = embedded.view(1, 1, -1)
        
        # print(output.shape, hidden.shape)
        # print(input.shape)
        # print(input)

        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# Defining the Decoder class
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)


    def forward(self, input, hidden):
        # reshape embedding vector as batch size=1, sequence length=1 and embedding size=embedding_size 
        embedded = self.embedding(input).view(1, 1, -1)
        output = F.relu(embedded)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# Attention class

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(" ")]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPairs(pair):
    in_tensor = tensorFromSentence(in_lang, pair[0])
    out_tensor = tensorFromSentence(out_lang, pair[1])
    return (in_tensor, out_tensor)    

In [ ]:
# define the train function

teacher_forcing_ratio = 0.5

def train(in_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    in_length = in_tensor.size(0)
    target_length = target_tensor.size(0)
    # print("in_length, target_length", in_length, target_length)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    
    for i in range(in_length):
        # print(in_tensor[i].shape, encoder_hidden.shape)
        encoder_output, encoder_hidden = encoder(in_tensor[i], encoder_hidden)
        encoder_outputs[i] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for i in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[i])
            decoder_input = target_tensor[i]    #teacher forcing

    else:
        for i in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  #detach from history as input     

            loss += criterion(decoder_output, target_tensor[i])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()/target_length



In [ ]:
# Helper function to keep tarck of time

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPairs(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

In [ ]:
# in_lang.word2index

In [ ]:
hidden_size = 256
encoder1 = Encoder(in_lang.n_words, hidden_size).to(device)
decoder1 = AttnDecoderRNN(hidden_size, out_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, decoder1, 125000, print_every=5000)

3m 17s (- 79m 9s) (5000 4%) 3.9083
6m 28s (- 74m 29s) (10000 8%) 3.0756
9m 41s (- 71m 1s) (15000 12%) 2.6686
12m 53s (- 67m 43s) (20000 16%) 2.4171
16m 7s (- 64m 28s) (25000 20%) 2.1872
19m 20s (- 61m 13s) (30000 24%) 1.9575
22m 34s (- 58m 3s) (35000 28%) 1.8147
25m 49s (- 54m 52s) (40000 32%) 1.6764
29m 3s (- 51m 39s) (45000 36%) 1.5600
32m 17s (- 48m 26s) (50000 40%) 1.4263
35m 31s (- 45m 13s) (55000 44%) 1.3296
38m 45s (- 41m 59s) (60000 48%) 1.2301
41m 59s (- 38m 45s) (65000 52%) 1.1642
45m 15s (- 35m 33s) (70000 56%) 1.0862
48m 31s (- 32m 20s) (75000 60%) 1.0267
51m 47s (- 29m 7s) (80000 64%) 0.9951
55m 2s (- 25m 54s) (85000 68%) 0.9480
58m 17s (- 22m 40s) (90000 72%) 0.8958
61m 32s (- 19m 26s) (95000 76%) 0.8741
64m 47s (- 16m 11s) (100000 80%) 0.8083
68m 2s (- 12m 57s) (105000 84%) 0.8163
71m 18s (- 9m 43s) (110000 88%) 0.7426
74m 33s (- 6m 28s) (115000 92%) 0.7682
77m 47s (- 3m 14s) (120000 96%) 0.7338
81m 1s (- 0m 0s) (125000 100%) 0.7438


In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(in_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(out_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=1000):
    # calculate the bleu score
    # gold_sents is a list of lists because every gold sentence may have multiple translations 
    gold_sents = []
    # pred_sents in a list
    pred_sents = []

    for i in range(n):
        pair = random.choice(pairs)
        output_words = evaluate(encoder, decoder, pair[0])
        gold_sents.append([pair[1]])

        output_sentence = ' '.join(output_words)
        pred_sents.append(output_sentence)

    bleu_score = sacre_bleu_score(pred_sents, gold_sents)
    print("BLEU Score = ", bleu_score, end='\n\n')

    for i in range(10):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
evaluateRandomly(encoder1, decoder1, 5000)

BLEU Score =  tensor(0.3398)

> we re winning .
= nous sommes en train de gagner .
< nous sommes en train de l embrasser . <EOS>

> i m skinny .
= je suis maigrichon .
< je suis maigrichonne . <EOS>

> she is a poor cook .
= c est une pietre cuisiniere .
< c est une mauvaise d elite . <EOS>

> he s leaning on a cane .
= il s appuie sur une canne .
< il s une une une . . <EOS>

> i m thirty years older than you .
= j ai trente ans de plus que vous .
< je ai trente ans de plus que vous . <EOS>

> they re disposable .
= ils sont jetables .
< ils sont jetables . <EOS>

> i am very busy these days .
= je suis tres occupe ces jours ci .
< je suis tres occupe ces jours ces jours . <EOS>

> you re alone aren t you ?
= vous etes seules n est ce pas ?
< tu es seule n est ce pas ? <EOS>

> i m looking forward to seeing you soon .
= j ai hate de te voir bientot .
< je me impatiente de te voir bientot . <EOS>

> i am very dangerous .
= je suis vraiment dangereux .
< je suis tres dangereux . <EOS>



In [ ]:
# define file paths to save and load the models
encoder_path = 'NMTencoder.pth'
decoder_path = 'NMTdecoder_Att.pth'

# save the encoder1 model
torch.save(encoder1.state_dict(), encoder_path)

# save the decoder1 model
torch.save(decoder1.state_dict(), decoder_path)

# load the saved models
# loaded_encoder = Encoder(in_lang.n_words, hidden_size).to(device)
# loaded_encoder.load_state_dict(torch.load(encoder_path))

# loaded_decoder = AttnDecoderRNN(hidden_size, out_lang.n_words, dropout_p=0.1).to(device)
# loaded_decoder.load_state_dict(torch.load(decoder_path))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# model_weights_path = '/content/drive/My Drive/NMT_Weights/my_model_weights.pt'
# torch.save(model.state_dict(), model_weights_path)


encoder_path = '/content/drive/My Drive/NMT_Weights/New_NMT_EngToGer_encoder.pth'
decoder_path = '/content/drive/My Drive/NMT_Weights/New_NMT_EngToGer_decoder_Att.pth'

# save the encoder1 model
torch.save(encoder1.state_dict(), encoder_path)

# save the decoder1 model
torch.save(decoder1.state_dict(), decoder_path)